# **0. Import the libraries you will need to use; do any other set ups**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#run this cell and we will speak about this in class
pd.set_option('display.float_format', lambda x: '%.1f' % x)

# **1. Read your datasets and check out whatever you wish in order to make sure that you know your data**

In [ ]:
dataframe1 = pd.read_excel('API_NY.GDP.MKTP.CD_DS2_en_excel_v2_4023900.xls', "2020")
dataframe1

In [ ]:
dataframe2 = pd.read_excel('resourcetradeearth_russia_all_wheat.xlsx', "Importers_2020")
dataframe2

In [ ]:
dataframe3 = pd.read_excel('resourcetradeearth_ukraine_all_wheat.xlsx', "Importers_2020")

dataframe3 = dataframe3.loc[:, ~dataframe3.columns.str.contains('^Unnamed')]

# **2. Based on your different datasets, create one single dataset to work with. This dataset should include only the variables we will really need:**

In [ ]:
dataframe1 = dataframe1.rename(columns={dataframe1.columns[0]:'country_name',
                         dataframe1.columns[1]:'country_code',
                         dataframe1.columns[3]:'pop'})

dataframe1

In [ ]:
dataframe2 = dataframe2.rename(columns={dataframe2.columns[1]:'country_code',
                         dataframe2.columns[2]:'importer',
                         dataframe2.columns[3]:'importer_region',
                         dataframe2.columns[6]:'value_1000usd_for_russia',
                         dataframe2.columns[7]:'weight_1000kg_for_russia'})

dataframe2

In [ ]:
dataframe3 = dataframe3.rename(columns={dataframe3.columns[1]:'country_code',
                         dataframe3.columns[2]:'importer',
                         dataframe3.columns[3]:'importer_region',
                         dataframe3.columns[6]:'value_1000usd_for_ukraine',
                         dataframe3.columns[7]:'weight_1000kg_for_ukraine'})

dataframe3

In [ ]:
dataframe4 = dataframe1[['country_code','GDP','pop']]
dataframe4

In [ ]:
dataframe5=dataframe2[['country_code','importer','importer_region','value_1000usd_for_russia','weight_1000kg_for_russia']]
dataframe5

In [ ]:
dataframe6=dataframe3[['country_code','importer','importer_region','value_1000usd_for_ukraine','weight_1000kg_for_ukraine']]
dataframe6

In [ ]:
dataframe7 = pd.merge(dataframe4, dataframe5, on='country_code', how='left')
dataframe7

In [ ]:
dataframe8 = pd.merge(dataframe7, dataframe6, on=['country_code','importer','importer_region'], how='left')
desired_order = ['country_code', 'importer', 'importer_region', 'value_1000usd_for_russia', 'value_1000usd_for_ukraine','weight_1000kg_for_russia','weight_1000kg_for_ukraine','GDP','pop']
dataframe9= dataframe8[desired_order]
dataframe9

In [ ]:
dataframe9[dataframe9.duplicated(keep=False)]
dataframe9 = dataframe9.reset_index().drop(['index'],axis=1)

dataframe9

# **3a. Display only rows that refer to Asia's imports from either Ukraine or Russia, without including China's imports**

In [ ]:
dataframe10 = dataframe9[(dataframe9.importer_region.str.contains('Asia',case=False))&(dataframe9.importer_region.str.contains('China',case=False))]
dataframe10

# **3b. How many rows are those?**

In [ ]:
row_count = len(dataframe10)
print(row_count)

# **4. How many total tones of wheat have been imported in Europe from both Ukraine and Russia?**

In [ ]:
europe_dataframe = dataframe9.loc[dataframe9['importer_region'] == 'Europe']
europe_dataframe 
europe_dataframe['weight_1000kg_for_russia'].sum() + europe_dataframe['weight_1000kg_for_ukraine'].sum()

# **5. Based on weight of total imports per capita, what are the top-10 wheat importers from both Ukraine and Russia? And what is the imported weight per capita in each top importer?**

In [ ]:
dataframe9['weight_per_capita'] = (dataframe9['weight_1000kg_for_russia'] + dataframe9['weight_1000kg_for_ukraine']) / dataframe9['pop']
top_10 = dataframe9.sort_values(by='weight_per_capita', ascending=False).head(10)

result_dataframe = top_10[['importer', 'weight_per_capita']]
result_dataframe.reset_index().drop(['index'],axis=1)

# **6. What is the total amount of money spent on wheat imports by region?**

In [ ]:
dataframe9['total_wheat_imports'] = dataframe9['value_1000usd_for_russia'] + dataframe9['value_1000usd_for_ukraine']

total_imports_by_region = dataframe9.groupby('importer_region')['total_wheat_imports'].sum()
total_imports_dataframe = total_imports_by_region.reset_index(name='Total Wheat Imports')
total_imports_dataframe